<a href="https://colab.research.google.com/github/mlnzlk/gpt/blob/master/dolly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#colab GPU 정보 확인
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
#@title 필요한 라이브러리 설치
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.5 MB/s eta 0:00:00
     ━━

In [ ]:
#@title 기본으로 깔려있는 pytorch 버전이 맞지않아 삭제하고 재설치
!pip uninstall -y torch
!pip uninstall -y torchvision
!pip -q install torch==1.13.1
!pip -q install torchvision==0.14.1

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
Found existing installation: torchvision 0.15.2+cu118
Uninstalling torchvision-0.15.2+cu118:
  Successfully uninstalled torchvision-0.15.2+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 91.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.12 requires torchvision>=0.8.2, which is not installed.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is inc

알파카 데이터셋은 직접 만든 학습이 되는지 테스트를 위해 사용한 것이고,
직접 데이터를 만들어서 사용할 때 는 생략

In [ ]:
#@title 알파카 데이터셋 로드
!git clone https://github.com/gururise/AlpacaDataCleaned.git

Cloning into 'AlpacaDataCleaned'...
remote: Enumerating objects: 747, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 747 (delta 64), reused 94 (delta 53), pack-reused 623
Receiving objects: 100% (747/747), 76.51 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (411/411), done.
Updating files: 100% (69/69), done.


In [ ]:
#@title 구글 클라우드 서비스 연결. Colab에서 세션이 만료되면 사라지는 파일을 관리하기 위해 사용. 만약 직접 파일을 로컬 컴퓨터에서 업로드한다면 구글 클라우드 관련 내용은 전부 생략 가능
from google.colab import auth
auth.authenticate_user()
!gcloud init

MessageError: ignored

In [ ]:
#@title 구글 클라우드 서비스안에 있는 버킷의 경로, 버킷은 디렉토리 같은 것
path_to_cloud_bucket = 'gs://gptneo-testdata230410/' #@param {type:"string"}

In [ ]:
#@title 구글 클라우스 서비스에서 데이터셋을 다운로드
from google.cloud import storage

bucket_name = 'gptneo-testdata230410'    # 서비스 계정 생성한 bucket 이름 입력
source_blob_name = 'instructions1000.jsonl'    # GCP에 저장되어 있는 파일 명
destination_file_name = './data/MyData.jsonl'    # 다운받을 파일을 저장할 경로("local/path/to/file")

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(source_blob_name)

blob.download_to_filename(destination_file_name)

In [ ]:
#@title 같은 데이터 천개로 구성된 데이터셋을 만들기 위해 사용했던 코드. 데이터셋이 있으면 생략 가능
import json

data_created = []
output_filename_created = "./data/MyData.jsonl"

for i in range (1,1000):
  data_created.append({'instruction':'"Action: Open Time: 2023-05-15 13:00 File: Microsoft Edge Day: Monday"\nthis is a log from what user doing\n.Give me pseudo code for the job that user was doing.','context':'','response': '# Open Microsoft Edge on Monday at 13:00\n\n# Check if it is Monday.\nif (today == "Monday"):\n\n  # Open Microsoft Edge.\n  open "Microsoft Edge"'})

with open(output_filename_created, 'w') as f:
    for data in data_created:
        f.write(json.dumps(data) + '\n')

In [ ]:
#@title 토크나이저 로드
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-2.8b") #사용할 코트나이저 이름 입력


# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id

data = load_dataset("json", data_files="./data/MyData.jsonl") #사용할 데이터셋


def generate_prompt(data_point):
    # taken from https://github.com/tloen/alpaca-lora
    if data_point["instruction"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### context:
{data_point["context"]}

### Response:
{data_point["response"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["response"]}"""


data = data.map(lambda data_point: {"prompt": tokenizer(generate_prompt(data_point))})

data

In [ ]:
#@title Finetuning을 위한 라이브러리들을 import해주는 코드 셀
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig

from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

In [ ]:
#@title 학습 설정
# Settings for A100 - For 3090
MICRO_BATCH_SIZE = 4  # change to 4 for 3090
#Batch Size는 한 번 가중치를 업데이트하기 위해 사용할 데이터의 개수. 64로 설정되어있으면 한 번 가중치를 업데이트하기 위해 64개의 데이터셋을 사용하는것
#데이터의 개수가 1000개이고 배치사이즈가 64이면 가중치 업데이트가 약 20번 일어남
BATCH_SIZE = 64
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # paper uses 3 전체 데이터셋을 학습할 횟수. 1000개의 데이터셋을 에포크 값 2로 학습하면 2000개의 데이터를 학습하는 것이다.
LEARNING_RATE = 2e-5  #학습할 때 가중치값의 변동률. 값이 클수록 학습할 때 가중치 값이 크게 변한다.
CUTOFF_LEN = 256
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [ ]:
#@title 토크나이저와 베이스 모델 로드
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-2.8b",
                                          add_eos_token=True,
                                          )

model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-2.8b",
                                  load_in_8bit=True,
                                  device_map="auto",
                                  )
tokenizer.pad_token_id = 0

model = prepare_model_for_int8_training(model, use_gradient_checkpointing=True)

In [ ]:
#@title 학습 설정2
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token



In [ ]:
#@title 데이터셋 로드
data = load_dataset("json", data_files="./data/MyData.jsonl")
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)

In [ ]:
data

In [ ]:
#@title 학습 시작
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="lora-dolly", #저장할 폴더명
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

model.save_pretrained("dolly-3b-1x1000") #저장할 모델명

In [ ]:
#@title 모델 실행을 위한 라이브러리 import, 토크나이저와 모델 import
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig

from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, PeftModel

tokenizerg = AutoTokenizer.from_pretrained("EleutherAI/pythia-2.8b",
                                          add_eos_token=True,
                                          )
tokenizerg.pad_token = tokenizerg.eos_token
tokenizerg.pad_token_id = tokenizerg.eos_token_id

modelg = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-2.8b",
                                  load_in_8bit=True,
                                  device_map="auto",
                                  )


modelg = PeftModel.from_pretrained(modelg, "dolly-3b-1x1000")

In [ ]:
#@title 모델 실행 코드 정의
from transformers import GenerationConfig

def dolly_generate(text):
    inputs = tokenizerg(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )

    print("Generating...")
    generation_output = modelg.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=128,
        pad_token_id = 0,
        eos_token_id = 50256
    )

    for s in generation_output.sequences:
        print(tokenizerg.decode(s))

In [ ]:
#@title PROMPT에 입력값 설정 후 실행, instruction에 질문, 지시사항 등이 들어가고 input에 추가 정보가 들어감. 테스트할 당시에는 추가정보로 로그 정보를 입력
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
'Action: Open, Time: 2023-05-15 13:00, File: Microsoft Edge, Day: Monday'
This is a log from what user doing.
Give me pseudo code for the job that user was doing.
### input:

### Response:"""

dolly_generate(PROMPT)